In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

import os
import pydicom as dcm

import scipy.signal

import astra


def add_gaussian_noise(proj_data, noise_level=0.01):
    # Generiamo il rumore eta
    np.random.seed(42)
    noise = np.random.normal(size=proj_data.shape) # random gaussian distribution
    noise /= np.linalg.norm(noise.flatten(), 2) # norma di frobenius
    noise *= noise_level * np.linalg.norm(proj_data.flatten(), 2)

    proj_data += noise

    return proj_data


def read_img(path):
    return np.load(path)


def get_name_list(path):
    import os
    n_list = []
    for patient in os.listdir(path):
        count = 0
        for slice_id in os.listdir(path + patient):
            n_list.append(path + patient + '/' + str(count) + '.npy')
            count += 1
    
    return n_list


def get_projection_operator():
    # create geometries and projector
    proj_geom = astra.create_proj_geom('fanflat', 1.0, 1024, np.linspace(0, np.pi, 60), 1800, 500)
    vol_geom = astra.create_vol_geom(512, 512)
    proj_id = astra.create_projector('cuda', proj_geom, vol_geom)

    print(astra.projector.is_cuda(proj_id))

    return astra.OpTomo(proj_id)

def get_output_path(path):
    out_path = './FACT30' + path[6:]

    return out_path


# Get the list of file name
n_list = get_name_list('./FDCT/')

# Get the projection operator defined above
A = get_projection_operator()

# Start Iterations!
for i, img_name in enumerate(n_list):
    print(i, '-', img_name)
    out_path = get_output_path(img_name)
    img = read_img(img_name).flatten()
    img = img / img.max()

    s = A*img
    noisy_s = add_gaussian_noise(s, 0.01)

    recon_img = A.reconstruct('FBP_CUDA', noisy_s)
    recon_img = recon_img.reshape((512, 512))
    
    np.save(out_path, recon_img)
    
# Visualize an example
plt.figure(figsize=(10, 10))
plt.imshow(recon_img)
plt.gray()
plt.show()


Bad key "text.kerning_factor" on line 4 in
/home/davide.evangelista/anaconda3/envs/tomo/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


True


"\n# Start Iterations!\nfor i, img_name in enumerate(n_list):\n    print(i, '-', img_name)\n    out_path = get_output_path(img_name)\n    img = read_img(img_name).flatten()\n    img = img / img.max()\n\n    s = A*img\n    noisy_s = add_gaussian_noise(s, 0.01)\n\n    recon_img = A.reconstruct('FBP_CUDA', noisy_s)\n    recon_img = recon_img.reshape((512, 512))\n    \n    np.save(out_path, recon_img)\n    \n# Visualize an example\nplt.figure(figsize=(10, 10))\nplt.imshow(recon_img)\nplt.gray()\nplt.show()\n"